# Custom Resources and Operators
> Explore the seldon-core custom resource and operator

- toc: true 
- badges: true
- comments: true
- categories: [kubernetes, docker]
- image: images/chart-preview.png

### Intro
In this post we will walk through the basics behind the seldon custom resource definition. 
At a high level, kubernetes only job is to maintain the desired state of the cluster. All interactions are changes to the desired state. The magic of kubernetes is that once you tell it the new state, it creates and maintains that state for you. 
The seldon-core projects serves inferences graphs on kubernetes with a custom resource and operator. These provide automation of complex systems, while allowing us to easily configure the desired state of our deployments without doing a lot of manual work. 


### Launch Cluster
To get started, let's get a cluster up and running. If you followed [part 1]() of this series, you can do so with kind. 
Below, I create a cluster, create a namespace called seldon-intro, then use `kubens` to make seldon-into my default namespace (so I don't have to include it in every command). 

In [1]:
!kind create cluster 
!kubectl create namespace seldon-intro
!kubens seldon-intro

Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.17.0) 🖼7l
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Have a question, bug, or feature request? Let us know! https://kind.sigs.k8s.io/#community 🙂


### KubeAPI, Custom Resources, and Operators
The KubeAPI is the medium through which all communication is handled in a kubernetes cluster. It is a rest server. When you send commands to a kubernetes cluster, you are hitting a specific api endpoint with commands for the server to execute. 
Kubernetes comes with some built-in objects you should be familiar with. Deployments, services, pods, etc. These objects are useful in and of themselves, but we often need to use many of them at once, which can get cumbersome. 
For those familiar with seldon, you know that you can create very complex inference graphs with many components. Instead of deploying and connecting all seldon services manually, we are able to build a single json/yaml configuration that deploys the entire graph. This is possible because of operators and custom resources. 
Operators and custom resources have an intimate relationship, and must be used in tandem. Earlier, I introduce the KubeAPI. We know that is how all internal and external communication is handled in the cluster. With operators, we extend the KubeAPI. In other words, operators allow us to add more endpoints to the KubeApi to carry out custom commands in our cluster. Which is where custom resources come in. They define the custom instructions for our new endpoint to execute. 
Let's see an operator and custom resource in action. 

### Install Seldon-core
I suggest using `helm` to install seldon-core. If you haven't used helm before, use [this page](https://helm.sh/) to find install instructions. (If you're on a mac, just use `brew install helm`). For those familiar with python, helm is like the pip for kubernetes. It works by using [helm charts](https://helm.sh/docs/topics/charts/). A chart is a group of files that describe a higher level application using built-in kubernetes resources. For example, you could use a helm chart to deploy a full stack web application. We are going to use helm to install seldon-core and seldon-core-operator. These helm charts are what will allow us to deploy seldon inference graphs. You can learn more about helm charts [here](https://helm.sh/docs/topics/charts/).

Once helm is installed, use it to install seldon-core and seldon-core-operator with the following command:

In [4]:
!helm install seldon-core seldon-core-operator \
    --repo https://storage.googleapis.com/seldon-charts \
    --set usageMetrics.enabled=true \
    --set ambassador.enabled=true \
    --namespace seldon-intro #unnecessary after using `kubens seldon-intro`, but keeping here to make sure the install is explicit

NAME: seldon-core
LAST DEPLOYED: Mon Jul 27 08:32:08 2020
NAMESPACE: seldon-intro
STATUS: deployed
REVISION: 1
TEST SUITE: None


After a successful install of seldon-core and seldon-core-operator, run the following command: 

In [5]:
!kubectl get deployments

NAME                        READY   UP-TO-DATE   AVAILABLE   AGE
seldon-controller-manager   1/1     1            1           6s


We see we now have a deployment, `seldon-controller-manager`, running in our seldon-intro namespace. 

In [7]:
!!kubectl describe svc seldon-webhook-service

['Name:              seldon-webhook-service',
 'Namespace:         seldon-intro',
 'Labels:            app=seldon',
 '                   app.kubernetes.io/instance=seldon-core',
 '                   app.kubernetes.io/managed-by=Helm',
 '                   app.kubernetes.io/name=seldon-core-operator',
 '                   app.kubernetes.io/version=1.2.1',
 'Annotations:       meta.helm.sh/release-name: seldon-core',
 '                   meta.helm.sh/release-namespace: seldon-intro',
 'Selector:          app.kubernetes.io/instance=seldon1,app.kubernetes.io/name=seldon,app.kubernetes.io/version=v0.5,app=seldon,control-plane=seldon-controller-manager',
 'Type:              ClusterIP',
 'IP:                10.96.113.42',
 'Port:              <unset>  443/TCP',
 'TargetPort:        443/TCP',
 'Endpoints:         10.244.0.5:443',
 'Session Affinity:  None',
 'Events:            <none>']

The docker image name used to create the seldon-controller-manager pod and deployment is called `docker.io/seldonio/seldon-core-operator:1.2.1`. That is because the seldon-controller-manager is the seldon-core-operator. 

Be sure to match the last line with the namespace you created in the previous command. We will talk more about what the other lines mean later in the series. 
After installing, run 

In [ ]:
!kubectl get pods -o wide

and you'll see you have a pod running with seldon-controller-manager in the name (followed by a random string, we will talk about why that is soon). Next, run 

In [ ]:
!kubectl get deployments

Seldon-controller-manager is a kubernetes operator. As described [here](https://www.jaegertracing.io/docs/1.16/operator/), kubernetes operators are extensions of the kubernetes api that allow users to easily package and deploy complex applications on kubernetes. The seldon-core-operator adds a lot of functionality to the kubernetes cluster and allows us to interact with seldon deployments as if they were a built-in kubernetes object. Along with this, we also installed the seldon-core custom resource definition. Kubernetes custom resources are extensions of the native kuberetes api. As you will see soon, we will now be able to deploy and interact with a new structure, the `sdep`, in the same fashion we deploy and monitor kubernetes deployments, servives, and other built-in resources. 


All interactions with a kubernetes cluster is the user specifying a new desired state, and the kubernetes cluster using it's resources to change into that desired state. When, a custom resource is created, an operator is needed as well to tell kubernetes how to handle updates to the desired state of the custom resource. 
The seldon-core operator is what allows users to make edits to currently running seldon deployments without downtime. 

Operators allows kubernetes to run stateful applications. A popular usecase for operators is databases. 

To get a better idea at how helm charts add to your kubernetes cluster, check out the [Exploring the Kubernetes API]() post digs into the basic internals of how your cluster actually receives commands. 

In [this post](https://enterprisersproject.com/article/2019/2/kubernetes-operators-plain-english?page=1), they write 
> If you had to sum up Kubernetes in a word, the best choice might not be “orchestration” but “automation.” That’s what it’s all about: Kubernetes enables the automation of the infrastructure (and corresponding operational burden of managing that infrastructure) necessary for running containerized applications – a must when running these apps at scale in production environments.

This is very evident with the Seldon deployment CRD and operator. As we move along and start to create our own seldon deployments, you will see the power of this automation. With a single file, a seldon deployment will launch multiple services, deployments, pods, and allow for continuous updates of all those components through the Kubernetes api. Seldon has leveraged the power of the automation offered by kubernetes to create inferences graphs of many components. 

Another great quote from that article: 
> “Operators are simplifying the process highly complex distributed database management by defining the installation, scale, updates, and management lifecycle of a stateful clustered application,” says Yossi Jana, DevOps team leader at AllCloud. From another vantage point, consider life without Operators. “Without Operators, many applications need intervention to deploy, scale, reconfigure, upgrade, or recover from faults,” Thompson says. “If your app – or apps that you depend on, such as your database management system – [requires] DevOps engineers hovering over a keyboard in these critical moments, hoping they get the steps correctly, you’re almost certain to have greater downtime and more stress in your team.”

From [kubernetes-operator-sdk tutorial](https://opensource.com/article/20/3/kubernetes-operator-sdk), operators are used to define custom resources. They extend the kubernetes api to tell the cluster how to handle those resources. 

Operators themselves run in pods. This is why you see the seldon-controller-manager deployment and pod running after we install seldon-core with helm. 

As described in the kubernetes docs [here](https://kubernetes.io/docs/concepts/extend-kubernetes/operator/), operators follow the controller pattern, which means they are responsible for keep the desired state of the custom resource they are responsible for. 
The seldon-core-operator is responsbile for the Seldon Deployment custom resource. That means, when we create or edit a seldon deployment, the seldon-core-operator is responsible for adjusting the kubernetes deployment to the desired state to meet the new edits applied by the user. 

In every article you read about kubernetes operators, you'll find some sentiment to the fact that is confusing the first time around. In fact, pretty much everything you learn about kubernetes will be confusing the first time around. Don't let that discourage you. That is why we are using seldon to help our understanding. Instead of reading description after description of what a custom resource and operator are, let's actually use them to begin to understand the power they provide. 

Let's launch our first seldon deployment onto the cluster to see what the consequences of creating a seldon deployment custom resource are. 

In [ ]:
%%bash
kubectl apply -f - << END
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: iris-model
spec:
  name: sklearn-iris-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/sklearn-iris:0.1
          imagePullPolicy: IfNotPresent
          name: sklearn-iris-classifier
    graph:
      children: []
      endpoint:
        type: REST
      name: sklearn-iris-classifier
      type: MODEL
    name: predictor
    replicas: 1
END

In [ ]:
!kubectl get pods